In [2]:
import requests
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer 
from Preprocessing.Preprocessing import Preprocessing_AR,Preprocessing_EN,split_by_language

[nltk_data] Downloading package punkt to C:\Users\Mohammad
[nltk_data]     Kher\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Mohammad
[nltk_data]     Kher\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Mohammad
[nltk_data]     Kher\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# get lawyers from database
lawyer_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/lawyers")
print(f"status code : {lawyer_response.status_code}")


# get rates from database
rate_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/rates")
print(f"status code : {rate_response.status_code}")


# get agencies from database
# agencie_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/agencies")
# print(f"status code : {agencie_response.status_code}")
agency_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/agencies?per_page=3&type=private&status=rejected")
print(f"status code : {agency_response.status_code}")


# get issues from database
issue_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/issues")
print(f"status code : {issue_response.status_code}")

status code : 200
status code : 200
status code : 200
status code : 200


In [12]:
lawyers_response = lawyer_response.json()
lawyers = pd.DataFrame(lawyers_response["data"]['lawyers']).rename(columns={
    "id":"lawyer_id"
}).drop(["email","union_number","affiliation_date","phone","agencies","issues","rates","avatar"],axis=1)
lawyers['years_of_experience'] = lawyers['years_of_experience'].apply(lambda x: f"{x}year")
lawyers.head()

,lawyer_id,name,address,union_branch,specializations,years_of_experience,description,rank
0,1,Ms. Michelle Auer MD,"635 Holden Drive\nConnhaven, MD 57130",Latakia,[arbitration],22year,Dolorem sint in amet quasi iure at. Et dicta n...,1.00
1,2,Dr. Mack Tremblay,"475 Collins Junction\nPort Zelmaberg, DE 54207",Homs,[penal],37year,Quos culpa doloremque sequi dicta. Quo eos qui...,0.00
2,3,Ardith Raynor,"420 Upton Drives Suite 393\nWillmsville, NH 32370",Damascus,[commercial],31year,Voluptas aut saepe nihil inventore aliquid cup...,1.67
3,4,Yvonne Ratke Jr.,"86057 Charlotte Ridges Suite 533\nRusseltown, ...",Homs,[terrorism],3year,Nemo et officia pariatur autem veritatis eum i...,3.00
4,5,Miller West,"11103 Homenick Garden Apt. 518\nIciehaven, ND ...",Damascus,[administrative],3year,Corporis aut laudantium rem saepe dolores vero...,2.00


In [13]:
rates_response = rate_response.json()
rates = pd.DataFrame(rates_response["data"]["rates"])
rates.dropna(inplace=True)
rates.drop(["id"],axis=1,inplace=True)
rates['rating_rate'] = rates['rating_rate'].apply(lambda x :f"{x}star")
rates.head()


,user_id,lawyer_id,rating_rate,review
0,60,133,4star,Et sit voluptate perferendis harum omnis volup...
2,136,165,3star,Eius id expedita dolor quidem delectus corpori...
4,143,165,4star,Fugit quasi similique laudantium voluptatem ut...
6,213,4,1star,Sit quam ut vel ab et eligendi dolorum corrupt...
9,108,13,4star,In sed facilis et rerum totam qui voluptas ven...


In [14]:
agencies_response = agency_response.json()
agencies = pd.DataFrame(agencies_response["data"]["agencies"]).rename(columns={
    "id":"agency_id"
})
agencies = agencies[["agency_id","lawyer_id"]]
agencies.head()

,agency_id,lawyer_id
0,39,97
1,43,125
2,49,116


In [15]:
issues_response = issue_response.json()
issues = pd.DataFrame(issues_response["data"]["issues"]).drop(["base_number","record_number","id","start_date","end_date","status"],axis=1)
issues.head()

,agency_id,court_name,type,estimated_cost,issue activity
0,73,شرعية,عائلية,775838,active
1,26,جزاء,مدنية,479780,active
2,102,جزاء,جنائية,4552262,in-active
3,49,جنايات,جنائية,5963450,active
4,29,شرعية,جنائية,325985,active


In [16]:
lawyers_with_rates = lawyers.merge(rates,on="lawyer_id")
lawyers_with_rates.head()

,lawyer_id,name,address,union_branch,specializations,years_of_experience,description,rank,user_id,rating_rate,review
0,4,Yvonne Ratke Jr.,"86057 Charlotte Ridges Suite 533\nRusseltown, ...",Homs,[terrorism],3year,Nemo et officia pariatur autem veritatis eum i...,3.0,213,1star,Sit quam ut vel ab et eligendi dolorum corrupt...


In [17]:
lawyers_with_rates= pd.merge(agencies,lawyers_with_rates,on=["lawyer_id"],how="inner")
lawyers_with_rates.head()

,agency_id,lawyer_id,name,address,union_branch,specializations,years_of_experience,description,rank,user_id,rating_rate,review
